# Genre classification by plot summary
The corpus contains descriptions of 30,000 films with high veriability. 
I assume that it has a high complexity and hidden semantics.
For instance, the word "death" can be at drama, horror, action, fight, and even comedy. 

Therefor, I choose to mimic semantic with **transformers**, and not just words counting/distributions/LDA.

Because this is an asymmetric semantic task, I used MSMARCO Models. 

Models trained with a causal language modeling (CLM) objective are better than BERT. 

# Model input: 
Embbedings vector for each plot (674 features). 

Few options: 1. Extract Sementic will be at the start and at the end of each plot. 2. slicing according to the model capacity, and feeding it separately. 3. slice + mix the vectorized representation. 


# Model output:
**Vector of logits to describe each class probability, per plot summary.**
This will give a feature representation, *instead* of one vs all. This is because different combinations of classes hide a different semantic. 

**Labels:** One hot encoding. (vector with size of: n_of_classes)

#imbalanced data
This data set is multy labeled and imbalanced. 

Adjustments relative to the imbalanced data need to be considered. {such as Random Undersampling (Tomek Link), Oversampling (SMOTE), Class weights in the models, Change Evaluation Metric and so on.}

#validation
For quantifing the performance, I need to find the best threshold (relative to accuracy) and fine tune it with validation set. 



In [1]:
pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.1 MB/s 
     |████████████████████████████████| 2.9 MB 9.5 MB/s 
     |████████████████████████████████| 3.3 MB 43.4 MB/s 
     |████████████████████████████████| 1.2 MB 51.2 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 895 kB 45.5 MB/s 
     |████████████████████████████████| 636 kB 59.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=1f8112ff9de2af238da99a295fb65c66ae0c30411226a7369a0317d95bb45c91
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torch.utils.data import Dataset
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/msmarco-distilroberta-base-v2')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
from google.colab import files
uploaded = files.upload()

Saving 100P.csv to 100P.csv


In [6]:
df = pd.read_csv("100P.csv", header=None)

In [7]:
# train-evaluation split: 
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.1)

In [8]:
class Net(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.out1 = (self.input_size + self.output_size)//2
        self.fc1 = torch.nn.Linear(in_features=self.input_size, out_features=self.out1)
        self.dropout = nn.Dropout(p=0.2)
        self.bn = nn.BatchNorm1d(self.out1)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(in_features=self.out1, out_features=self.output_size)
        self.act = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        # x = self.bn(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.act(x)
        return x

    def predict(self, instance, label):
        embeddings = model.encode(instance)

In [9]:
embeddings_init = [model.encode(df.iloc[0][0])]

input_size = len(embeddings_init[0])
output_size = train_df.iloc[0, 1:].shape[0]
net = Net(input_size=input_size, output_size=output_size).to(device)


# I'm using the nn.BCELoss for multiclass-multilabled classification. 
BCELoss and ***not*** BCEWithLogitsLoss because I want to use the same model for validation/testing.

In [10]:
criterion = nn.BCELoss(reduction='mean')
optimizer = optim.Adam(net.parameters(), lr=0.0001)


# Creating batchs

# Training and Evaluation
Due to time constraints, I only tried to catch the overfiting point.

Unfortunately I did not had time to add:
- Data loader
- Creating Batches
- Randomization
- Revaluation of the Validation_LOSS with Batch and not just single item
- BN
- Evaluation matrix 

In [11]:
    # eval_embeddings = [model.encode(eval_df.iloc[i][0]) for i in range(eval_df.shape[0])]

    # input_size = len(embeddings[0])
    # output_size = train_df.iloc[0, 1:].shape[0]
    # net = Net(input_size=input_size, output_size=output_size).to(device)


In [12]:
# idx_val_batch = np.random.choice(list(range(eval_df.shape[0])), size = 10)
# val_b_embeddings = [model.encode(val_df.iloc[i][0]) for i in idx_val_batch]
# val_b_embeddings = torch.tensor(val_b_embeddings)

In [13]:
# train_b_embed = model.encode(train_df.iloc[1][0])


In [14]:
train_df.shape[0]

80

In [45]:
epochs=10
batch_size = 5 
for epoch in range(epochs):
    train_df.sample(frac=1)
    val_df.sample(frac=1)
    loss = torch.tensor(0.)
    
    for i in range(0, train_df.shape[0], batch_size):
        # print("i is: ", i)
        train_batch = train_df.iloc[i:i+batch_size]
        loss = torch.FloatTensor([0])
        for j in range(train_batch.shape[0]):
            train_embed = torch.tensor(model.encode(train_batch.iloc[j][0]))
            l = train_batch.iloc[j, 1:].values
            labels = torch.tensor(l.astype(float)).to(device)
            optimizer.zero_grad()
            outputs = net(train_embed.to(device))
            labels = labels.float()
            outputs = outputs.float()
            loss = loss + criterion(outputs.unsqueeze(-1), labels.unsqueeze(-1))
        loss.backward()
        optimizer.step()

    if epoch%3==0:
        val_loss = torch.tensor(0.)
        for k in range(0, val_df.shape[0], batch_size):
            val_batch = val_df.iloc[k:k+batch_size]
            loss = torch.FloatTensor([0])
            for m in range(val_batch.shape[0]):
                val_embed = torch.tensor(model.encode(val_batch.iloc[j][0]))
                val_l = val_batch.iloc[m, 1:].values
                val_labels = torch.tensor(val_l.astype(float)).to(device)
                optimizer.zero_grad()
                val_outputs = net(val_embed.to(device))
                val_labels = labels.float()
                val_outputs = val_outputs.float()
                val_loss += criterion(val_outputs.unsqueeze(-1), val_labels.unsqueeze(-1))
  
        torch.save(net.state_dict(), "model_weights_"+str(epoch))
        torch.save(model.state_dict(), 'checkpoint.pth'+str(epoch))
        files.download('checkpoint_'+str(epoch)+'.pth')

        print('Epoch [%d/%d], Iter [%d]:' %(epoch+1, epochs, i+1))
        print('                        Train loss: %.4f' % (loss))
        print('                        Evaluation loss: %.4f' % (val_loss))



FileNotFoundError: ignored

In [ ]:
# torch.save(net.state_dict(), "model_weights.pth")


# Creating an inference class

In [ ]:
# I created a dictionary for the Genres names at the preprocessing notebook. 
# lis = [df2.keys()[i] for i in range(df2.shape[1])]
# for i, gen in enumerate (lis):
#     print (i,':"',gen,'",')

In [43]:
genre_dict = {0 :" Absurdism ",
1 :" Acid western ",
2 :" Action ",
3 :" Action Comedy ",
4 :" Action Thrillers ",
5 :" Action/Adventure ",
6 :" Addiction Drama ",
7 :" Adult ",
8 :" Adventure ",
9 :" Adventure Comedy ",
10 :" Airplanes and airports ",
11 :" Albino bias ",
12 :" Alien Film ",
13 :" Alien invasion ",
14 :" Americana ",
15 :" Animal Picture ",
16 :" Animals ",
17 :" Animated Musical ",
18 :" Animated cartoon ",
19 :" Animation ",
20 :" Anime ",
21 :" Anthology ",
22 :" Anthropology ",
23 :" Anti-war ",
24 :" Anti-war film ",
25 :" Apocalyptic and post-apocalyptic fiction ",
26 :" Archaeology ",
27 :" Archives and records ",
28 :" Art film ",
29 :" Auto racing ",
30 :" Avant-garde ",
31 :" B-Western ",
32 :" B-movie ",
33 :" Backstage Musical ",
34 :" Baseball ",
35 :" Beach Film ",
36 :" Beach Party film ",
37 :" Bengali Cinema ",
38 :" Biker Film ",
39 :" Biographical film ",
40 :" Biography ",
41 :" Biopic [feature] ",
42 :" Black comedy ",
43 :" Black-and-white ",
44 :" Blaxploitation ",
45 :" Bloopers & Candid Camera ",
46 :" Bollywood ",
47 :" Boxing ",
48 :" Breakdance ",
49 :" British Empire Film ",
50 :" British New Wave ",
51 :" Bruceploitation ",
52 :" Buddy Picture ",
53 :" Buddy cop ",
54 :" Buddy film ",
55 :" Business ",
56 :" Camp ",
57 :" Caper story ",
58 :" Cavalry Film ",
59 :" Chase Movie ",
60 :" Chick flick ",
61 :" Childhood Drama ",
62 :" Children's ",
63 :" Children's Entertainment ",
64 :" Children's Fantasy ",
65 :" Children's/Family ",
66 :" Chinese Movies ",
67 :" Christian film ",
68 :" Christmas movie ",
69 :" Clay animation ",
70 :" Cold War ",
71 :" Combat Films ",
72 :" Comdedy ",
73 :" Comedy ",
74 :" Comedy Thriller ",
75 :" Comedy Western ",
76 :" Comedy film ",
77 :" Comedy horror ",
78 :" Comedy of Errors ",
79 :" Comedy of manners ",
80 :" Comedy-drama ",
81 :" Coming of age ",
82 :" Coming-of-age film ",
83 :" Computer Animation ",
84 :" Computers ",
85 :" Concert film ",
86 :" Conspiracy fiction ",
87 :" Costume Adventure ",
88 :" Costume Horror ",
89 :" Costume drama ",
90 :" Courtroom Comedy ",
91 :" Courtroom Drama ",
92 :" Creature Film ",
93 :" Crime ",
94 :" Crime Comedy ",
95 :" Crime Drama ",
96 :" Crime Fiction ",
97 :" Crime Thriller ",
98 :" Cult ",
99 :" Culture & Society ",
100 :" Cyberpunk ",
101 :" Czechoslovak New Wave ",
102 :" Dance ",
103 :" Demonic child ",
104 :" Detective ",
105 :" Detective fiction ",
106 :" Disaster ",
107 :" Docudrama ",
108 :" Documentary ",
109 :" Dogme 95 ",
110 :" Domestic Comedy ",
111 :" Doomsday film ",
112 :" Drama ",
113 :" Dystopia ",
114 :" Ealing Comedies ",
115 :" Early Black Cinema ",
116 :" Education ",
117 :" Educational ",
118 :" Ensemble Film ",
119 :" Environmental Science ",
120 :" Epic ",
121 :" Epic Western ",
122 :" Erotic Drama ",
123 :" Erotic thriller ",
124 :" Erotica ",
125 :" Escape Film ",
126 :" Essay Film ",
127 :" Existentialism ",
128 :" Experimental film ",
129 :" Exploitation ",
130 :" Expressionism ",
131 :" Extreme Sports ",
132 :" Fairy tale ",
133 :" Family & Personal Relationships ",
134 :" Family Drama ",
135 :" Family Film ",
136 :" Family-Oriented Adventure ",
137 :" Fan film ",
138 :" Fantasy ",
139 :" Fantasy Adventure ",
140 :" Fantasy Comedy ",
141 :" Fantasy Drama ",
142 :" Feature film ",
143 :" Female buddy film ",
144 :" Feminist Film ",
145 :" Fictional film ",
146 :" Filipino ",
147 :" Filipino Movies ",
148 :" Film ",
149 :" Film & Television History ",
150 :" Film adaptation ",
151 :" Film noir ",
152 :" Film à clef ",
153 :" Film-Opera ",
154 :" Filmed Play ",
155 :" Finance & Investing ",
156 :" Foreign legion ",
157 :" Future noir ",
158 :" Gangster Film ",
159 :" Gay ",
160 :" Gay Interest ",
161 :" Gay Themed ",
162 :" Gay pornography ",
163 :" Gender Issues ",
164 :" Giallo ",
165 :" Glamorized Spy Film ",
166 :" Goat gland ",
167 :" Gothic Film ",
168 :" Graphic & Applied Arts ",
169 :" Gross out ",
170 :" Gross-out film ",
171 :" Gulf War ",
172 :" Hagiography ",
173 :" Hardcore pornography ",
174 :" Haunted House Film ",
175 :" Health & Fitness ",
176 :" Heaven-Can-Wait Fantasies ",
177 :" Heavenly Comedy ",
178 :" Heist ",
179 :" Hip hop movies ",
180 :" Historical Documentaries ",
181 :" Historical Epic ",
182 :" Historical drama ",
183 :" Historical fiction ",
184 :" History ",
185 :" Holiday Film ",
186 :" Horror ",
187 :" Horror Comedy ",
188 :" Horse racing ",
189 :" Humour ",
190 :" Hybrid Western ",
191 :" Illnesses & Disabilities ",
192 :" Indian Western ",
193 :" Indie ",
194 :" Inspirational Drama ",
195 :" Instrumental Music ",
196 :" Interpersonal Relationships ",
197 :" Inventions & Innovations ",
198 :" Japanese Movies ",
199 :" Journalism ",
200 :" Jukebox musical ",
201 :" Jungle Film ",
202 :" Juvenile Delinquency Film ",
203 :" Kafkaesque ",
204 :" Kitchen sink realism ",
205 :" LGBT ",
206 :" Language & Literature ",
207 :" Latino ",
208 :" Law & Crime ",
209 :" Legal drama ",
210 :" Libraries and librarians ",
211 :" Linguistics ",
212 :" Live action ",
213 :" Malayalam Cinema ",
214 :" Marriage Drama ",
215 :" Martial Arts Film ",
216 :" Master Criminal Films ",
217 :" Media Satire ",
218 :" Media Studies ",
219 :" Medical fiction ",
220 :" Melodrama ",
221 :" Mockumentary ",
222 :" Mondo film ",
223 :" Monster ",
224 :" Monster movie ",
225 :" Movie serial ",
226 :" Movies About Gladiators ",
227 :" Mumblecore ",
228 :" Music ",
229 :" Musical ",
230 :" Musical Drama ",
231 :" Musical comedy ",
232 :" Mystery ",
233 :" Mythological Fantasy ",
234 :" Natural disaster ",
235 :" Natural horror films ",
236 :" Nature ",
237 :" Neo-noir ",
238 :" Neorealism ",
239 :" New Hollywood ",
240 :" News ",
241 :" Ninja movie ",
242 :" Northern ",
243 :" Operetta ",
244 :" Outlaw ",
245 :" Outlaw biker film ",
246 :" Parkour in popular culture ",
247 :" Parody ",
248 :" Patriotic film ",
249 :" Period piece ",
250 :" Pinku eiga ",
251 :" Plague ",
252 :" Point of view shot ",
253 :" Political Documetary ",
254 :" Political cinema ",
255 :" Political drama ",
256 :" Political satire ",
257 :" Political thriller ",
258 :" Pornographic movie ",
259 :" Pornography ",
260 :" Pre-Code ",
261 :" Prison ",
262 :" Prison escape ",
263 :" Prison film ",
264 :" Private military company ",
265 :" Propaganda film ",
266 :" Psycho-biddy ",
267 :" Psychological horror ",
268 :" Psychological thriller ",
269 :" Punk rock ",
270 :" Race movie ",
271 :" Reboot ",
272 :" Religious Film ",
273 :" Remake ",
274 :" Revenge ",
275 :" Revisionist Fairy Tale ",
276 :" Revisionist Western ",
277 :" Road movie ",
278 :" Road-Horror ",
279 :" Roadshow theatrical release ",
280 :" Roadshow/Carny ",
281 :" Rockumentary ",
282 :" Romance Film ",
283 :" Romantic comedy ",
284 :" Romantic drama ",
285 :" Romantic fantasy ",
286 :" Romantic thriller ",
287 :" Samurai cinema ",
288 :" Satire ",
289 :" School story ",
290 :" Sci Fi Pictures original films ",
291 :" Sci-Fi Adventure ",
292 :" Sci-Fi Horror ",
293 :" Sci-Fi Thriller ",
294 :" Science Fiction ",
295 :" Science fiction Western ",
296 :" Screwball comedy ",
297 :" Sex comedy ",
298 :" Sexploitation ",
299 :" Short Film ",
300 :" Silent film ",
301 :" Silhouette animation ",
302 :" Singing cowboy ",
303 :" Slapstick ",
304 :" Slasher ",
305 :" Slice of life story ",
306 :" Social issues ",
307 :" Social problem film ",
308 :" Softcore Porn ",
309 :" Space opera ",
310 :" Space western ",
311 :" Spaghetti Western ",
312 :" Splatter film ",
313 :" Sponsored film ",
314 :" Sports ",
315 :" Spy ",
316 :" Stand-up comedy ",
317 :" Star vehicle ",
318 :" Statutory rape ",
319 :" Steampunk ",
320 :" Stoner film ",
321 :" Stop motion ",
322 :" Superhero ",
323 :" Superhero movie ",
324 :" Supermarionation ",
325 :" Supernatural ",
326 :" Surrealism ",
327 :" Suspense ",
328 :" Swashbuckler films ",
329 :" Sword and Sandal ",
330 :" Sword and sorcery ",
331 :" Sword and sorcery films ",
332 :" Tamil cinema ",
333 :" Teen ",
334 :" Television movie ",
335 :" The Netherlands in World War II ",
336 :" Therimin music ",
337 :" Thriller ",
338 :" Time travel ",
339 :" Tokusatsu ",
340 :" Tollywood ",
341 :" Tragedy ",
342 :" Tragicomedy ",
343 :" Travel ",
344 :" Vampire movies ",
345 :" War film ",
346 :" Werewolf fiction ",
347 :" Western ",
348 :" Whodunit ",
349 :" Women in prison films ",
350 :" Workplace Comedy ",
351 :" World History ",
352 :" World cinema ",
353 :" Wuxia ",
354 :" Z movie ",
355 :" Zombie Film "}

In [44]:
class Test():
    def __init__(self, plot):
        self.plot = plot

    def inference(self):

        embedTest = model.encode(self.plot)
        embedTest = torch.tensor(embedTest)
        net_pred = Net(input_size=768, output_size=356).to(device)
        net_pred.load_state_dict(torch.load('/content/model_weights.pth', map_location='cpu'))
        net_pred.eval()
        outputs = net_pred(embedTest.to(device))
        out ={}
        for i, prob in enumerate(outputs):
            if prob>0.1:
                prob =prob*100
                # print('Genre' ,genre_dict[i], 'prob [%.2f]:' %(prob))
                print(genre_dict[i], "(%.0f" %(prob), "%)")
                # out[i] = prob
                # print('Epoch [%d/%d], Iter [%d]:' %(epoch+1, epochs, i+1))
        # return out

In [ ]:
plot= "The film begins in Kuvukiland, an undeveloped kingdom somewhere in Africa. Mr. Bones arrives as a baby, the sole survivor of an airplane crash that happens nearby. He grows older and becomes the bone-throwing prophesier for the kingdom. King Tsonga, ruler of Kuvukiland, longs for a male child to be heir to the throne. After having seventeen children, all of them girls, he gives up hope, until he remembers fathering a boy decades before in Sun City. He immediately sends Mr. Bones to find the future prince. At the same time golf star Vince \"The Prince\" Lee, along with his coach, The Wild Bull, arrives in Sun City for a golf tournament. A local casino owner places a huge bet on Vince Lee winning the tournament, but just before it begins, Wild Bull is injured in a freak accident. He quickly recovers, but is held against his will in a local hospital. Without his coach, Vince Lee plays terribly, until he meets the eccentric Mr. Bones, who believes him to be the actual prince, and gives him a lucky streak. Vince nearly wins the game until Mr. Bones remembers his mission and stops a perfect putt. Vince retires in disgrace, but meets a local singer, Laleti, afterwards, whom he is stricken with. Mr. Bones notices this, and by impersonating her, he kidnaps Vince. The next day, Wild Bull manages to escape from the hospital and goes on a search for Vince, along with Laleti and her mother. Enraged by Vince Lee's performance, and by the fact that everyone had gone missing, the casino owner mounts a search for them in a helicopter, along with two of his henchmen. After a series of comical mishaps, they all meet near Kuvukiland. Mr. Bones introduces Vince to King Tsonga, but after discovering that Vince is terrified of animals, King Tsonga disowns him, and prepares to die. The casino owner quickly locates Vince and Laleti, and attempts to kill them both, but Vince escapes. He finds Laleti tied to a tree with a lion about to eat her, and overcoming his fear, he chases the lion away. King Tsonga sees this, and decides that he doesn't want to die. Soon after, the casino owner reappears, but Mr. Bones, with the help of an elephant, causes the helicopter to crash. King Tsonga proclaims that Vince is his son, but askes Mr. Bones to throw his prophesy bones once more just to be sure. As Mr. Bones does this, Wild Bull arrives upon the scene, and it is confirmed that he is the actual prince. The film closes with Mr. Bones, King Tsonga, and Wild Bull watching from Kuvukiland as Vince Lee, now married to Laleti, wins the US Open golf tournament.\n"

test = Test(plot)
test.inference()


In [ ]:
from google.colab import drive
drive.mount('/model_weights')

In [ ]:
class KNN():
    def __init__(self, plot):
        self.plot = plot

    def inference(self):

        embedTest = model.encode(self.plot)
        embedTest = torch.tensor(embedTest)
        net_pred = Net(input_size=768, output_size=356).to(device)
        net_pred.load_state_dict(torch.load('/content/model_weights.pth', map_location='cpu'))
        net_pred.eval()
        outputs = net_pred(embedTest.to(device))
        out ={}
        for i, prob in enumerate(outputs):
            if prob>0.1:
                prob =prob*100
                # print('Genre' ,genre_dict[i], 'prob [%.2f]:' %(prob))
                print(genre_dict[i], "(%.0f" %(prob), "%)")
                # out[i] = prob
                # print('Epoch [%d/%d], Iter [%d]:' %(epoch+1, epochs, i+1))
        # return out